# Курс "Программирование на языке Python. Уровень 4. Анализ и визуализация данных на языке Python. Библиотеки numpy, pandas, matplotlib"

## Модуль 6. Агрегирование и групповые операции в pandas

1. Группировка данных
2. Агрегирование данных
3. Групповые операции над данными
4. Групповые операции и статистический анализ
5. Сводные таблицы
6. Визуализация данных средствами pandas



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 10.0)
%matplotlib inline

### Группировка данных

Под группировкой будем понимать разделение объекта pandas на части по одному или нескольким признакам. Такое объединение данных применяется для:
 - вычисления групповых статстик (минимум, максимум, сумма, и т.д.)
 - применения групповых преобразований
 - вычисления сводных таблиц и кросстабулирования

В pandas для группировки используется функция ```groupby()```. Эта функция сама по себе ничего не вычисляет,кроме промужуточных данных о ключах, по которым производится объедиение. Функция возвращает объект, у которого есть методы для агрегатных вычислений.
 

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [ ]:
grouped.mean()

Для группировки можно использовать несколько объектов Series:

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

...или выполнять группировку по всему DataFrame'у, передав ключи в виде списка:

In [ ]:
# это выражение должно вывести тот же результат, что и выражение выше
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

...также можно указать один ключ. Обратите внимание: куда пропал ключ 'key2'?

In [ ]:
means = df.groupby('key1').mean()
means

Группировку можно проводить по любой оси. Например, сгруппируем данные по типу:

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)
grouped.size()

Объект SeriesGroupBy поддерживает итерирование. Так, группы можно обходить в цикле ```for```. При этом итератор возвращает значение ключа и объект DataFrame группы:

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

__ЗАДАНИЕ__ Сгруппируйте датафрейм df по признакам key1 и key2 и обратите внимание, что вернет первый параметр итератора.

In [ ]:
# ваш код здесь


Также для группировки можно использовать словари и массивы. Рассмотрим пример:

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Alex', 'Jim', 'Travis'])
people

In [ ]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [ ]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

Также можно использовать для группировки функцию (например, ```len()```):

In [ ]:
people.groupby(len).sum()

#### Множественный индекс

Вспомним группировку из предыдущего примера. Обратите внимание на индекс:

In [ ]:
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

DataFrame с множественным индексом можно задать явно:

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    ['specialized', 'santa cruz', 'gt', 'fuji', 'ninjago']],
                                    names=['cty', 'brand'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

Для группировки указываем уровень в параметре ```level=```.

In [ ]:
hier_df.groupby(level='brand', axis=1).sum()

### Агрегирование данных

Список оптимизированных агрегатных функций pandas:

- ```count()``` - количество отличных от NaN значений в группе
- ```sum()``` - сумма
- ```mean()``` - среднее (также по отличным от NaN значениям)
- ```median()``` - медиана (тоже по отличным от NaN)
- ```var()``` - дисперсия
- ```min()``` - минимум
- ```max()``` - максимум
- ```prod()``` - произведение
- ```first()``` - первый отличный от NaN в группе
- ```last()``` - последний отличный от NaN в группе

Также можно написать свою собственную функцию и передать ее в функцию ```agg()```. Она будет принимать на вход объект Series и возращать скалярное значение.


In [ ]:
df

In [ ]:
def mean_median(arr):
    return arr.mean() - arr.median()
df.groupby('key1').agg(mean_median)

Также в ```agg()``` можно передавать список функций, стандартные функции передаются по именам:

In [ ]:
df.groupby('key1').agg(['mean', 'median', mean_median])

In [ ]:
df.groupby('key1').agg([("Среднее", 'mean'), ("Медиана", 'median'), ("Среднее-медиана", mean_median)])

__ЗАДАНИЕ__: Для набора данных "чаевые" посчитайте следующее:
 - общую сумму чаевых за каждый день
 - средний процент для групп в зависимости от пола, а потом и дня
 - выведите минимальный и максимальный процент чаевых в зависимости от дня и времени

In [ ]:
tips = pd.read_csv('data/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

In [ ]:
# ваш код здесь
tips.groupby('day')['tip'].sum()

In [ ]:
tips.groupby(['sex', 'day'])['tip_pct'].mean()

### Групповые операции над данными

Функция apply позволяет применить заданную функцию к каждой группе. И более того, она может вернуть векторное значение.

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]
top(tips, n=6)

Так мы можем вывести самых щедрых на чаевые из групп "курящие/некурящие":

In [ ]:
tips.groupby('smoker').apply(top)

Параметры к нашей функции можно передать через ту же функцию ```apply()```

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

Избавиться от индекса в выводе данных можно с помощью параметра ```group_keys=False```

In [ ]:
tips.groupby(['smoker', 'day'], group_keys=False).apply(top, n=1, column='total_bill')

__ЗАДАНИЕ__: В заданном ниже датасете посчитайте средний балл по фамилиям студентов:

In [ ]:
students = ['Вася Иванов', 'Витя Петров', 'Таня Текслер', 'Игорь Иванов' \
            , 'Петр Текслер', 'Игорь Иванов', 'Андрей Иванов', 'Ольга Петрова']
marks = [4, 4, 5, 3, 4, 5, 5, 3]
df_marks = pd.DataFrame( {'student': students, 'mark': marks} )

# ваш код здесь

### Групповые операции при статистическом анализе

Пример для вычисления группового взвешенного среднего.

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
def get_wavg(g):
    return np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

Кореляция.

In [ ]:
close_px = pd.read_csv('data/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.head()

In [ ]:
rets = close_px.pct_change().dropna() # сбросим незаполненные данные и посчитаем изменения в процентном отношении
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Сводные таблицы

Для построения сводных таблиц используется метод ```pivot_table()```. По умолчанию она считает средние по каждому показателю.

Рассмотрим на примере "чаевых":

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['day', 'time'],
                 columns='smoker', margins=True, fill_value=0)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

__ЗАДАНИЕ__: постройте сводную таблицу с суммами чаевых в зависимости от дня недели, времени и пола.

In [ ]:
# ваш код здесь


#### Кросстабуляция

или таблица сопряженности - частный случай сводной таблицы для подсчета групповых частот.

In [ ]:
party_counts = pd.crosstab(tips['day'], tips['size'])# Not many 1- and 6-person parties
print(party_counts)

__ЗАДАНИЕ__ Реализуйте ту же задачу, но функцией ```pivot_table()```.

In [ ]:
# ваш код здесь

__Визуализация данных средствами Pandas__

Рассмотрим базовые функции визуализации в pandas на примере датасета моделей playboy с 1953 по 2009 года.

In [ ]:
df_playboy = pd.read_csv('data/girls.csv')
df_playboy.head()

In [ ]:
pd.plotting.scatter_matrix(df_playboy[['Bust', 'Waist', 'Hips', 'Height', 'Weight']], 
                  figsize=(15, 15))
plt.show()

In [ ]:
df_playboy['Height'].hist(bins=20)

In [ ]:
df_playboy.boxplot(column='Waist', by='Month')

Желающие могут ознакомиться с примером анализа данных - решением задачи поиска аномалий в этом датасете, в статье Ю. Кашницкого на "Хабре": https://habr.com/ru/post/251225/

__ПРАКТИКА__

Загрузите датасет "toy_budget.csv". Он содержит информацию по доходам и расходам подразделений компании по месяцам. Прямой доход имеет параметр "Type" = "Income". Внутренние расходы компании имеют тип "Costs".

В продолжении задания из предыдущего модуля, выведите суммы прямого дохода за год по каждому подразделению и по каждому месяцу.

Также выведите суммы внутренних расходов компании.

Следующая задача будет состоять в том, чтобы реализовать распределение внутренних расходов компании среди "зарабатыващих" подразделений (признак "Div"). По правилам компании, они должны распределяться пропорционально  ежемесячным доходам подразделений.

Для начала добавьте в dataframe строки с распределениями расходов, которые соответствуют следующим требованиям:
 - они привязаны к зарабатывающим подразделениям ("Div" соответствуют "Div" соответствующих подразделений), 
 - "Type" = "Intercompany"
 - "Account" = "Intercompany Cost"
 - суммы по месяцам соответствуют общим расходам компании, распределенным по "зарабатывающим" подразделениям пропорционально их доходам в данном месяце, взятым со знаком "минус".

То есть в сумме по месяцам это распределение должно полностью компенсироваться позициями с типом "Costs".

Следующий шаг - разбейте эти добавленные позиции на расходы по каждой статье их "Costs" и добавьте их с типом "Intercompany", "Account" - соответствует  статье из "Account" распределяемых записей типа "Costs".

Проверьте себя, суммы по всему датафрейму должны соответствовать доходам компании ("Type" = "Income").

Как обычно в этом курсе, избегайте использования циклов!

_Подсказка:_ вычисляйте распределения на базе матрицы весовых коэффициентов.


In [ ]:
months = ['apr',
 'may',
 'jun',
 'jul',
 'aug',
 'sep',
 'oct',
 'nov',
 'dec',
 'jan',
 'feb',
 'mar',]

df_toy_budget = pd.read_csv('data/toy_budget.csv', sep=';', index_col=0)
df_toy_budget

In [ ]:
df_budget_pivoted = ?
df_budget_pivoted_income = ?

In [ ]:
# выведите суммы здесь

In [ ]:
# выполните рассчет распределения всех расходов здесь

In [ ]:
# выполните рассчет распределения расходов по каждой статье здесь

In [ ]:
# проверьте себя здесь